In [135]:
library(BGVAR) #load library
library(ggplot2)
library(zoo)
library(janitor) #rowtonames function

## Load Data

In [227]:
# Read in the data files
df_covid = read.csv('../../data/tidy/cases_deaths_concat.csv')
df_google = read.csv('../../data/tidy/google_activity.csv')
df_interv = read.csv('../../data/tidy/govt_interventions.csv')
df_deter = read.csv('../../data/tidy/deterministic.csv', row.names = 1 )

In [228]:
head(df_covid)

,X,iso,X2.15.2020,X2.16.2020,X2.17.2020,X2.18.2020,X2.19.2020,X2.20.2020,X2.21.2020,X2.22.2020,⋯,X8.22.2020,X8.23.2020,X8.24.2020,X8.25.2020,X8.26.2020,X8.27.2020,X8.28.2020,X8.29.2020,X8.30.2020,X8.31.2020
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,cases,ALB,0,0,0,0,0,0,0,0,⋯,8275,8427,8605,8759,8927,9083,9195,9279,9380,9513
2,cases,ARE,8,9,9,9,9,9,9,13,⋯,66617,67007,67282,67621,68020,68511,68901,69328,69690,70231
3,cases,ARG,0,0,0,0,0,0,0,0,⋯,336802,342154,350867,359638,370188,380292,392009,401239,408426,417735
4,cases,AUS,15,15,15,15,15,15,15,15,⋯,24811,24915,25053,25204,25322,25448,25547,25670,25746,25819
5,cases,AUT,0,0,0,0,0,0,0,0,⋯,25062,25253,25495,25706,26033,26361,26590,26985,27166,27438
6,cases,BEL,1,1,1,1,1,1,1,1,⋯,81468,81936,82092,82447,83030,83500,83952,84599,85042,85236


In [229]:
# Here, we make all the names uniform
# Ideally, this should be done in the preprocessing script in python (make this a TODO)
names(df_covid)[3:201] <- gsub(x = names(df_covid)[3:201], pattern = "X", replacement = "")
names(df_google)[3:201] = names(df_covid)[3:201]
names(df_interv)[3:201] = names(df_covid)[3:201]
df_covid = cbind(df_covid[['iso']], df_covid[['X']], df_covid[,3:201]) #reorder ISO and Variable columns

In [230]:
# Change the variable names to be shorter
# As I suggested, come up with 3- or 4-letter abbreviations for ALL the variables 
# Save these new names in the Python pre-processing script. Once this is done, the next few lines should be obsolete
df_interv$category <- gsub(x = df_interv$category, pattern = "Public health measures", replacement = "phm")
df_interv$category <- gsub(x = df_interv$category, pattern = "Governance and socio-economic measures", replacement = "sem")
df_interv$category <- gsub(x = df_interv$category, pattern = "Social distancing", replacement = "sd")
df_interv$category <- gsub(x = df_interv$category, pattern = "Movement restrictions", replacement = "mr")
df_interv$category <- gsub(x = df_interv$category, pattern = "Lockdown", replacement = "ld")
df_interv$category <- gsub(x = df_interv$category, pattern = "Humanitarian exemption", replacement = "he")

In [231]:
head(df_google) #check structure here to ensure uniformity

,iso,activity,2.15.2020,2.16.2020,2.17.2020,2.18.2020,2.19.2020,2.20.2020,2.21.2020,2.22.2020,⋯,8.22.2020,8.23.2020,8.24.2020,8.25.2020,8.26.2020,8.27.2020,8.28.2020,8.29.2020,8.30.2020,8.31.2020
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ARE,grocery,4,4,1,1,0,1,2,2,⋯,-9,1,-10,-8,-7,-5,-11,-5,2,-3
2,ARE,parks,5,4,5,5,4,6,6,4,⋯,-43,-37,-42,-41,-41,-42,-55,-46,-38,-40
3,ARE,residential,1,1,1,1,1,1,1,1,⋯,8,22,10,10,10,11,10,8,9,10
4,ARE,retail,0,1,-1,-2,-2,-2,-3,-2,⋯,-23,-19,-25,-24,-23,-22,-30,-23,-18,-21
5,ARE,transit,0,1,1,0,-1,1,0,-2,⋯,-44,-53,-42,-41,-41,-41,-47,-42,-40,-41
6,ARE,workplaces,2,2,2,2,2,1,-1,3,⋯,-15,-58,-20,-21,-20,-20,-13,-11,-21,-22


In [232]:
plot(seq(3,231),df_google[1,3:231]) #quick exploratory plotting
plot(seq(4,231),matrixStats::rowDiffs(as.matrix(df_google[1,3:231]))) #quick exploratory plotting

ERROR: Error in `[.data.frame`(df_google, 1, 3:231): undefined columns selected


In [233]:
plot(seq(4,231),matrixStats::rowDiffs(as.matrix(df_covid[1,3:231]))) #quick exploratory plotting
plot(seq(5,231),matrixStats::rowDiffs(as.matrix(df_covid[1,3:231]), differences=2L)) # twice differenced
plot(seq(3,231),df_covid[1,3:231]) #quick exploratory plotting

ERROR: Error in `[.data.frame`(df_covid, 1, 3:231): undefined columns selected


In [234]:
df_covid[, 5:201] = matrixStats::rowDiffs(as.matrix(df_covid[, 3:201]), differences=2L) # convert to daily cases and then difference
df_google[, 4:201] = matrixStats::rowDiffs(as.matrix(df_google[, 3:201])) # 1st difference of google activity/mobility
# More data transformations can/may need to be done here. 

In [235]:
as = read.csv('')

Warning message in file(file, "rt"):
“file("") only supports open = "w+" and open = "w+b": using the former”


ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : no lines available in input


In [236]:
#Standardize names of first 2 columns (This can/should also be done in python pre-processing script)
names(df_covid)[1:2] = c('country', 'variable')
names(df_google)[1:2] = c('country', 'variable')
names(df_interv)[1:2] = c('country', 'variable')

In [241]:
length(unique(df_google$country))
length(unique(df_covid$country))
length(unique(df_interv$country))

[1] 56

[1] 56

[1] 61

In [202]:
sort(unique(df_interv$country))

[1] "ALB" "ARE" "ARG" "AUS" "AUT" "BEL" "BGR" "BRA" "CAN" "CHE" "CHL" "COL"
[13] "CZE" "DEU" "DNK" "EGY" "ESP" "EST" "FIN" "FRA" "GBR" "GRC" "HKG" "HRV"
[25] "HUN" "IDN" "IND" "IRL" "ISL" "ISR" "ITA" "JPN" "KHM" "KOR" "LTU" "LUX"
[37] "LVA" "MAR" "MEX" "MYS" "NLD" "NOR" "NZL" "PHL" "POL" "PRT" "ROU" "RUS"
[49] "SAU" "SGP" "SRB" "SVK" "SVN" "SWE" "THA" "TUR" "UKR" "URY" "USA" "VNM"
[61] "ZAF"

In [205]:
sort(unique(df_google$country))

[1] "ARE" "ARG" "AUS" "AUT" "BEL" "BGR" "BRA" "CAN" "CHE" "CHL" "COL" "CZE"
[13] "DEU" "DNK" "EGY" "ESP" "EST" "FIN" "FRA" "GBR" "GRC" "HKG" "HRV" "HUN"
[25] "IDN" "IND" "IRL" "ISR" "ITA" "JPN" "KHM" "LTU" "LUX" "LVA" "MAR" "MEX"
[37] "MYS" "NLD" "NOR" "NZL" "PHL" "POL" "PRT" "ROU" "SAU" "SGP" "SVK" "SVN"
[49] "SWE" "THA" "TUR" "UKR" "URY" "USA" "VNM" "ZAF"

In [240]:
unique(df_interv$country)[!unique(df_interv$country) %in% unique(df_covid$country)]
unique(df_interv$country)[!unique(df_interv$country) %in% unique(df_google$country)]
unique(df_covid$country)[!unique(df_covid$country) %in% unique(df_google$country)]
unique(df_interv$country)[!unique(df_interv$country) %in% unique(df_deter$iso)]

[1] "HKG" "KOR" "RUS" "SRB" "USA"

[1] "ALB" "ISL" "KOR" "RUS" "SRB"

[1] "ALB" "ISL"

[1] "BGR" "HKG" "ROU"

In [239]:
df_endo <- rbind(df_covid[,1:201], df_google[,1:201]) #endogenous variables
df_exo <- df_interv[,1:201] #strictly exogenous variables

### Convert endogenous data into list of matrices

In [151]:
#Convert endogenous data into list of dataframes; each list element corresponds to country
# For each country, rows = time obs; columns = endogenous variables
endo = list()
for (i in unique(df_endo$country)) {
    country_df = janitor::row_to_names(t(df_endo[df_endo$country==i, 2:201]), 1)
    rownames(country_df) <- c()
  #head(b_df)
  endo[[i]] <- country_df 
}

In [130]:
names(endo)

[1] "AFG" "AGO" "ALB" "AND" "ARE" "ARG" "ARM" "ATG" "AUS" "AUT" "AZE" "BDI"
 [13] "BEL" "BEN" "BFA" "BGD" "BGR" "BHR" "BHS" "BIH" "BLR" "BLZ" "BOL" "BRA"
 [25] "BRB" "BTN" "BWA" "CAF" "CAN" "CHE" "CHL" "CHN" "CIV" "CMR" "COL" "COM"
 [37] "CRI" "CUB" "CYP" "CZE" "DEU" "DJI" "DMA" "DNK" "DOM" "DZA" "ECU" "EGY"
 [49] "ERI" "ESH" "ESP" "EST" "ETH" "FIN" "FJI" "FRA" "GAB" "GBR" "GEO" "GHA"
 [61] "GIN" "GMB" "GNB" "GNQ" "GRC" "GRD" "GTM" "GUY" "HND" "HRV" "HTI" "HUN"
 [73] "IDN" "IND" "IRL" "IRQ" "ISL" "ISR" "ITA" "JAM" "JOR" "JPN" "KEN" "KGZ"
 [85] "KHM" "KWT" "LBN" "LBR" "LCA" "LIE" "LKA" "LSO" "LTU" "LUX" "LVA" "MAR"
 [97] "MCO" "MDG" "MDV" "MEX" "MLI" "MLT" "MNE" "MNG" "MOZ" "MRT" "MUS" "MWI"
[109] "MYS" "NAM" "NER" "NGA" "NIC" "NLD" "NOR" "NPL" "NZL" "OMN" "PAK" "PAN"
[121] "PER" "PHL" "PNG" "POL" "PRT" "PRY" "QAT" "ROU" "RWA" "SAU" "SDN" "SEN"
[133] "SGP" "SLE" "SLV" "SMR" "SOM" "SSD" "STP" "SUR" "SVK" "SVN" "SWE" "SYC"
[145] "TCD" "TGO" "THA" "TJK" "TLS" "TTO" "TUN" "TUR" "UGA" "UKR" "URY" "UZB"
[157] "VCT" "VEN" "VNM" "YEM" "ZAF" "ZMB" "ZWE" "SLB" "ABW" "CPV" "HKG" "PRI"
[169] "USA"

### Convert exogenous data into list of matrices

In [153]:
exo = list()
for (i in unique(df_exo$country)) {
    country_df = janitor::row_to_names(t(df_exo[df_exo$country==i, 2:201]), 1)
    rownames(country_df) <- c()
  #head(b_df)
  exo[[i]] <- country_df 
}

In [154]:
# Create uniform weight matrix (static in the current implementation)
bW = data.frame(matrix(0, ncol = length(endo), nrow = length(endo)))
rownames(bW) = names(endo)
colnames(bW) = names(endo)
for ( i in seq(1, length(endo))) {
  for (j in seq(1, length(endo))) {
    if (i != j) {
      #print(c(i,j))
      bW[i,j] = 1.0/(length(endo) - 1)
    }
  }
}

# Check weight matrix structure
all(colnames(bW)==names(endo)[1:(length(endo)-1)])
rowSums(bW)
diag(as.matrix(bW))
bW = as.matrix(bW)

# convert matrix into list as required
bWlist <- list()
for (i in colnames(bW)) {
  w_df <- data.frame(bW[,c(i)])
  colnames(w_df) <- i
  rownames(w_df) <- rownames(bW)
  bWlist[[i]] <- w_df
}

Warning message in colnames(bW) == names(endo)[1:(length(endo) - 1)]:
“longer object length is not a multiple of shorter object length”


[1] FALSE

ALB ARE ARG AUS AUT BEL BGR BRA CAN CHE CHL COL CZE DEU DNK EGY ESP EST FIN FRA 
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 
GBR GRC HRV HUN IDN IND IRL ISL ISR ITA JPN KHM LTU LUX LVA MAR MEX MYS NLD NOR 
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 
NZL PHL POL PRT ROU SAU SGP SVK SVN SWE THA TUR UKR URY VNM ZAF HKG USA 
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

ALB ARE ARG AUS AUT BEL BGR BRA CAN CHE CHL COL CZE DEU DNK EGY ESP EST FIN FRA 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
GBR GRC HRV HUN IDN IND IRL ISL ISR ITA JPN KHM LTU LUX LVA MAR MEX MYS NLD NOR 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
NZL PHL POL PRT ROU SAU SGP SVK SVN SWE THA TUR UKR URY VNM ZAF HKG USA 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

In [155]:
length(bWlist)

[1] 58

In [ ]:
## BGVAR
# SSVS prior
Hyperparm.ssvs <- list(tau0   = 0.1,  # coefficients: prior variance for the spike
                       # (tau0 << tau1)
                       tau1   = 3,    # coefficients: prior variance for the slab  
                       # (tau0 << tau1)
                       kappa0 = 0.1,  # covariances: prior variance for the spike 
                       # (kappa0 << kappa1)
                       kappa1 = 7,    # covariances: prior variance for the slab 
                       # (kappa0 << kappa1)
                       a_i    = 0.01, # prior for the shape parameter of the IG
                       b_i    = 0.01, # prior for the scale parameter of the IG
                       p_i    = 0.5,  # prior inclusion probability of coefficients
                       q_ij   = 0.5   # prior inclusion probability of covariances
)

model <- bgvar(Data = endo, #endogenous variables
               Ex = exo, # exogenous variables
               W = bWlist, #static weight matrix (use uniform weights)
               plag=7,
               saves=100, burns=100, prior="SSVS", SV=TRUE, hyperpara=Hyperparm.ssvs, 
               h = 292, #40% held out 
               thin = 1,
               variable.list=NULL,
               trend = FALSE,
               save.country.store=FALSE
)